In [35]:
import pandas as pd
import os
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer

## Load data and forward pass

In [47]:
DATA_FOLDER = 'data'
MODEL_FOLDER = 'models'
path = 'ACHIEVEMENT.csv'
train_split = .8
df = pd.read_csv(os.path.join(os.getcwd(), DATA_FOLDER, path))
df_train = df.sample(frac=train_split, random_state=0)
df_test = df.drop(df_train.index).reset_index(drop=True)

In [37]:
VALUES = ['ACHIEVEMENT', 'BENEVOLENCE', 'CONFORMITY', 'HEDONISM', 'POWER', 'SECURITY', 'SELF-DIRECTION', 'STIMULATION', 'TRADITION', 'UNIVERSALISM']

class ValueDataset(Dataset):
    
    def __init__(self, tokenizer, df): 
        self.scenarios = df['scenario'].values.tolist()
        self.N = df.shape[0]

        inp = tokenizer(self.scenarios, return_tensors='pt', padding=True, truncation=True)
        self.input_ids = inp.get('input_ids')
        self.attention_mask = inp.get('attention_mask')
        self.token_type_ids = inp.get('token_type_ids')
        self.target = df['label'].values.tolist()
    
    def __getitem__(self, index):
        return self.input_ids[index], self.attention_mask[index], self.token_type_ids[index], self.target[index]

    def __len__(self):
        return self.N

In [86]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('prajjwal1/bert-small', num_labels = 1)

train_dataset = ValueDataset(tokenizer, df_train)
test_dataset = ValueDataset(tokenizer, df_test)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [87]:
(test_dataloader)

171

## Training loop

In [28]:
from torch.nn import MSELoss, Tanh

# Assuming model is already defined and loaded as in your code
loss_fn = MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
tanh = Tanh()

epochs = 200
epoch_loss = []

for e in tqdm(range(epochs)):
    model.train()  # Set the model to training mode
    batch_loss = []

    for input_ids, attention_mask, token_type_ids, targets in train_dataloader:
        # Forward pass
        optimizer.zero_grad()  # Clear gradients
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        logits = outputs.logits
        logits = tanh(logits).squeeze(-1)  # Apply tanh activation to bound outputs

        # Calculate loss
        loss = loss_fn(logits, targets.float())  # Make sure targets are floats
        batch_loss.append(loss.item())

        # Backward pass
        loss.backward()
        optimizer.step()

    # Calculate and print the average loss for the epoch
    avg_loss = np.mean(batch_loss)
    epoch_loss.append(avg_loss)
    tqdm.write(f'Epoch {e+1}/{epochs}, Loss: {avg_loss}')

# Print the loss after all epochs
print(epoch_loss)


  0%|          | 1/200 [00:16<54:20, 16.38s/it]

Epoch 1/200, Loss: 0.5997630135304686


  1%|          | 2/200 [00:33<56:14, 17.04s/it]

Epoch 2/200, Loss: 0.4668541535519172


  2%|▏         | 3/200 [00:48<52:28, 15.98s/it]

...

Epoch 198/200, Loss: 0.0026587110289677885


100%|█████████▉| 199/200 [48:46<00:15, 15.05s/it]

Epoch 199/200, Loss: 0.0028117373244047326


100%|██████████| 200/200 [49:01<00:00, 14.71s/it]

Epoch 200/200, Loss: 0.002899042176199989
[0.5997630135304686, 0.4668541535519172, 0.3946911189435185, 0.33546844815862353, 0.2841189795214197, 0.23198208374821622, 0.21820684101270593, 0.1681743427247241, 0.15145094719701918, 0.13406356083958046, 0.14118144591001497, 0.1150017509209937, 0.10545471747932227, 0.08475685637930165, 0.08123993673834248, 0.06970191827934721, 0.07586110834086286, 0.06474312705298264, 0.05759649798004092, 0.06193342135436293, 0.054157888219840286, 0.0508661035961215, 0.05101783485219315, 0.044596257147149765, 0.047763248986524086, 0.04238380548422751, 0.043172559120516846, 0.04165002719863602, 0.03969760822213214, 0.035623029414294426, 0.029680487375868404, 0.03372379000960053, 0.03454692482921309, 0.03340187912428941, 0.027913269772450778, 0.029520464494176533, 0.026323313855201654, 0.028602539305237755, 0.028244951739907265, 0.028024526501911274, 0.02603353997287543, 0.026228947149913594, 0.0277674147096611, 0.026513087873657543, 0.02600482932926304, 0.0220

## Testing

In [89]:
from sklearn.metrics import f1_score, recall_score, accuracy_score

def get_predictions():
    model.eval()
    with torch.no_grad():
        for data in tqdm(iter(test_dataloader)):
            input_ids, attention_mask, token_types_ids, targets = data
            inp = {'input_ids' : input_ids, 'attention_mask' : attention_mask, 'token_type_ids' : token_types_ids}
            output = model(**inp)
            logits = output.logits
            predictions = tanh(logits).squeeze(-1)

            return predictions, targets

In [95]:
model = torch.load(os.path.join(os.getcwd(), MODEL_FOLDER, 'bert_achievenemt_finetuned_1.pt'))

predictions, targets = get_predictions()

  0%|          | 0/1 [00:00<?, ?it/s]


In [100]:
predictions

tensor([ 0.7111,  0.3014, -0.1772, -0.9845,  0.3819, -0.7010,  0.9651,  0.8240,
        -0.9924, -0.0203, -0.5209, -0.9924,  0.5603, -0.0478, -0.9897, -0.9885,
        -0.0386, -0.9899, -0.9684, -0.9909,  0.0157, -0.0327, -0.2532, -0.9894,
        -0.9805,  0.0594, -0.5274, -0.0174, -0.9465, -0.3620, -0.1607, -0.0396,
        -0.1207, -0.0403, -0.9914,  0.0590, -0.0224, -0.4950, -0.0397,  0.9047,
        -0.0497,  0.4768, -0.8985, -0.0495, -0.9853, -0.9833, -0.7307,  0.1535,
        -0.9081, -0.9512, -0.9905, -0.0721, -0.9934, -0.9882, -0.9269,  0.9787,
        -0.9856, -0.8335, -0.2841, -0.9167, -0.0183,  0.1737, -0.9943,  0.4433,
        -0.9860, -0.8799, -0.9916, -0.9899, -0.9869, -0.9776, -0.9925, -0.5803,
        -0.0145,  0.0096, -0.9911, -0.6890, -0.9203, -0.9878, -0.5461, -0.9840,
         0.0052, -0.9824, -0.2332, -0.0239, -0.9939, -0.8248,  0.3475, -0.1884,
        -0.9440, -0.9849, -0.0393, -0.1398, -0.0495, -0.0300, -0.5077,  0.0191,
         0.3301,  0.1692,  0.9925, -0.55

In [107]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def regression_to_classification_metrics(predictions, targets):
    # Define thresholds for class assignment based on regression predictions
    thresholds = [-0.5, 0.5]

    # Convert regression predictions to discrete labels [-1, 0, 1]
    labels = np.digitize(predictions, bins=thresholds)

    # Calculate precision, recall, and F1 scores
    accuracy = accuracy_score(targets, labels)
    precision = precision_score(targets, labels, average='weighted')
    recall = recall_score(targets, labels, average='weighted')
    f1 = f1_score(targets, labels, average='weighted')

    return accuracy, f1, recall, precision

# Example usage:
# predictions, targets = get_predictions()
accuracy, f1, recall, precision = regression_to_classification_metrics(predictions = predictions, targets = targets)
print("F1 Score:", f1)
print("Recall Score:", recall)
print("Precision Score:", precision)

F1 Score: 0.22234156820622983
Recall Score: 0.2573099415204678
Precision Score: 0.20230223343398437


/opt/homebrew/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## OLD

In [ ]:
epochs = 30
optimizer = torch.optim.Adam(model.parameters())
loss_fn = torch.nn.functional.mse_loss
batch_loss = []
epoch_loss = []

for e in tqdm(range(epochs)):
    for data in train_dataloader:
        # Unpack the data from the dataloader
        input_ids, attention_mask, token_types_ids, targets = data
        
        # Clear previous gradients
        optimizer.zero_grad()

        # Prepare inputs for the model
        inp = {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_types_ids
        }

        # Forward pass
        output = model(**inp)
        loss = loss_fn(output.logits, targets.float().unsqueeze(1))

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Store loss
        batch_loss.append(loss.item())

    # Calculate mean loss for the epoch
    epoch_loss.append(np.mean(batch_loss))
    batch_loss = []

    # Print the mean loss for the epoch
    tqdm.write(f'Epoch {e+1}/{epochs}, Loss: {epoch_loss[-1]}')

# Print the loss after all epochs
print(epoch_loss)
